# Nutrition Notebook

This notebook defines functions and queries for working with recommended daily intake of various nutrients.

## Linear Optimization Functions

The following functions help to construct and solve a linear optimization problem using nutrition data. We assume the data is in the form of a dataframe where each row represents a food item, and columns represent macronutrients, micronutrients, and other facts. The constraint matrix will be built in the same format, meaning that the actual constraints are the *columns,* not the *rows,* of the constraint matrix. In other words, the matrix we build is actually the transpose of the matrix we want.

### initialize_constraints_with_rdi(data)

#### Usage

```constraints, bounds = initialize_constraints_with_rdi(data)```

#### Description

Takes a dataframe whose rows represent food items, and having columns named `pct_*` that represent percentages of the Recommended Dietary Intake (RDI) of one or more nutrients, and returns two things: a dataframe of linear constraints, and an array of upper bounds for each. Assuming that your objective involves meeting the RDI, these can be used as the basis of a linear optimization by adding more constraints or by defining the desired objective function.

In [1]:
def initialize_constraints_with_rdi(data, days=1):
    # Fields named like pct_foo are assumed to mean "percent of RDI for nutrient foo"
    nutrients = [ field for field in data.columns if field[:4] == 'pct_' ]

    # Require 100% of every nutrient with an RDA. Since the optimizer treats
    # constraints as "less than," we need to negate everything
    constraints = np.multiply( data.loc[:, nutrients].to_numpy(), -1 )
    bounds = [ -1 * days for row in constraints.T ]

    return constraints, bounds

### add_constraint(data, constraints=[], bounds=[], nutrient=None, min=None, max=None, days=1)

Takes the given constraints and bounds, and adds new constraints for the specified nutrient. If `min` is specified, a constraint is added requiring that the total for `nutrient` must be at least `min`. Similarly if `max` is specified, a constraint is added requiring that the total be no more than `max`.

If nutrient isn't specified, or if both `max` and `min` are `None`, then the constraints are returned unchanged.

In [2]:
# Function for adding constraints to the nutrition LP
def add_constraint(data, constraints=[], bounds=[], nutrient=None, min=None, max=None, days=1):
    if not nutrient:
        return(constraints, bounds)
    if nutrient not in data.columns:
        return(constraints, bounds)

    coefs = np.transpose([data.loc[:, nutrient].to_numpy()])
    
    if min is not None:
        constraints = np.c_[constraints, np.multiply(coefs, -1)]
        bounds.append(-1 * min * days)
    
    if max is not None:
        constraints = np.c_[constraints, coefs]
        bounds.append(max * days)
    
    return(constraints, bounds)

### add_weight_constraint(data, constraints=[], bounds=[], regex=r'', limits=[], min=None, max=None)

This method adds a constraint on the `min` and/or `max` weight for all food items matching `regex`. It then returns the updated `constraints` and `bounds`. The `min` and `max` weight are specified in grams.

The purpose of `limits` is to keep track of per-food-item limits, in case the user wants to disallow some food items completely.

In [3]:
# Function for adding a constraint on the total weight of specified foods
def add_min_weight(data, constraints=[], bounds=[], regex=r'', limits=[], min=None, max=None):
    matched = data['food_code'].astype(str).str.match(regex)
    limits = [(0,max/100) if matched[i] else limits[i] for i in range(len(matched))]
    
    coefs = np.transpose([1 if matched[i] else 0 for i in range(len(matched))])
    
    # Minimum...
    if min:
        constraints = np.c_[constraints, -1 * coefs]
        bounds.append(-1 * min / 100)

    # Maximum...
    if max:
        constraints = np.c_[constraints, +1 * coefs]
        bounds.append(+1 * max / 100)
    
    return(constraints, bounds, limits)

## Query Functions

This section defines functions that return SQL queries related to nutrients and recommended daily intake.

### query_food_percent_rdi(age=None, sex=None)

In [4]:
def query_food_percent_rdi(age=None, sex=None):
    if not (age and sex):
        return None

    return("""
        SELECT * FROM contrib.food_dri_pct_view
        WHERE sex = '%s'
        AND   type = 'recommended'
        AND   age_from = (
                SELECT MAX(age_from)
                FROM contrib.food_dri_pct_view
                WHERE age_from <= %s AND sex = '%s'
              )
    """ % (sex, age, sex))


### query_nutrient_tolerable_upper_limits(age=None, sex=None)

In [5]:
def query_nutrient_tolerable_upper_limits(age=None, sex=None):
    if not (age and sex):
        return None

    return("""
        SELECT
            tagname, (tolerable_upper.amount / rec.amount) AS pct_tolerable_upper
        FROM (
                SELECT age_from, sex, tagname, amount
                FROM dietary_reference_intake.dietary_reference_intake
                WHERE type = 'tolerable_upper'
                AND age_from = (
                    SELECT MAX(age_from)
                    FROM dietary_reference_intake.dietary_reference_intake
                    WHERE age_from <= %s
                    AND sex = '%s'
                )
                AND sex = '%s'
        ) AS tolerable_upper
        JOIN dietary_reference_intake.dietary_reference_intake AS rec USING(age_from, sex, tagname)
        WHERE
            rec.type = 'recommended'
            AND (tolerable_upper.amount / rec.amount) > 1
    """ % (age, sex, sex))


